In [1]:
import os
import json
import torch
import torchtext as text
import sys
import tqdm
from datasets import load_dataset
import pandas as pd
import string
from torchtext.data import get_tokenizer
from vocab import *
from utils import *

from tqdm.notebook import tqdm

In [ ]:
dataset = load_dataset("squad")

train_dic = {'passage': [], 'question': [], 'answer': []}
for i in range(len(dataset['train'])):
    datum = dataset['train'][i]
    for j in range(len(datum['answers']['text'])):
        train_dic['passage'].append(datum['context'])
        train_dic['question'].append(datum['question'])
        train_dic['answer'].append(datum['answers']['text'][j])

train = pd.DataFrame(train_dic)

val_dic = {'passage': [], 'question': [], 'answer': []}
for datum in dataset['validation']:
    for elem in datum['answers']['text']:
        ans_id = 0
        val_dic['passage'].append(datum['context'])
        val_dic['question'].append(datum['question'])
        val_dic['answer'].append(elem)

val = pd.DataFrame(val_dic)

In [ ]:
for i in tqdm(range(len(train))):
    row = train.iloc[i]
    passage = clean_text(row['passage'].lower())
    question = clean_text(row['question'].lower())
    answer = clean_text(row['answer'].lower())
    
    train.iloc[i]['passage'] = passage
    train.iloc[i]['question'] = question
    train.iloc[i]['answer'] = answer
    
for i in tqdm(range(len(val))):
    row = val.iloc[i]
    passage = clean_text(row['passage'].lower())
    question = clean_text(row['question'].lower())
    answer = clean_text(row['answer'].lower())
    
    val.iloc[i]['passage'] = passage
    val.iloc[i]['question'] = question
    val.iloc[i]['answer'] = answer

In [ ]:
train.to_csv('./data/train.csv', index=False)
val.to_csv('./data/dev.csv', index=False)

In [2]:
vocab = build_vocab()

Building Vocabulary
Saved the vocab.
